# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

mongoimport --type json -d uk_food -c establishments --drop --jsonArray establishments.json

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'config', 'local', 'uk_food']


In [4]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [5]:
# review the collections in our new database
print(db.list_collection_names())

['establishments']


In [8]:
# assign the collection to a variable
establishments = db['establishments']

In [9]:
# review a document in the establishments collection
document = establishments.find_one()
pprint(document)

{'AddressLine1': 'The Bay',
 'AddressLine2': 'St Margarets Bay',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'The Coastguard Inn',
 'BusinessType': 'Pub/bar/nightclub',
 'BusinessTypeID': 7843,
 'ChangesByServerID': 0,
 'Distance': 4587.347174863443,
 'FHRSID': 1034540,
 'LocalAuthorityBusinessID': 'PI/000078691',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6DY',
 'RatingDate': '2022-08-17T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('65f75b080d813317aabf2eb7'),
 'geocode': {'latitude': '51.152225', 'longitude': '1.387974'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1034540',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extractDate': '0001-01-01T0

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [10]:
# Create a dictionary for the new restaurant data
new_restaurant = {
    "BusinessName": "Penang Flavours",
    "BusinessType": "Restaurant/Cafe/Canteen",
    "BusinessTypeID": "",
    "AddressLine1": "Penang Flavours",
    "AddressLine2": "146A Plumstead Rd",
    "AddressLine3": "London",
    "AddressLine4": "",
    "PostCode": "SE18 7DY",
    "Phone": "",
    "LocalAuthorityCode": "511",
    "LocalAuthorityName": "Greenwich",
    "LocalAuthorityWebSite": "http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress": "health@royalgreenwich.gov.uk",
    "scores": {
        "Hygiene": "",
        "Structural": "",
        "ConfidenceInManagement": ""
    },
    "SchemeType": "FHRS",
    "geocode": {
        "longitude": "0.08384000",
        "latitude": "51.49014200"
    },
    "RightToReply": "",
    "Distance": 4623.9723280747176,
    "NewRatingPending": True
}

In [12]:
# Insert the new restaurant into the collection
db = mongo['uk_food']
collection = db['establishments']
insert_result = collection.insert_one(new_restaurant)

In [14]:
# Check that the new restaurant was inserted
if insert_result.inserted_id:
    print("New restaurant inserted successfully!")
else:
    print("Failed to insert new restaurant.")

New restaurant inserted successfully!


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [17]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = {
    "BusinessType": "Restaurant/Cafe/Canteen"
}

projection = {
    "_id": 0,
    "BusinessTypeID": 1,
    "BusinessType": 1
}

result = collection.find_one(query, projection)

if result:
    print("BusinessTypeID:", result["BusinessTypeID"])
    print("BusinessType:", result["BusinessType"])
else:
    print("No document found for the given BusinessType.")

BusinessTypeID: 1
BusinessType: Restaurant/Cafe/Canteen


3. Update the new restaurant with the `BusinessTypeID` you found.

In [18]:
# Update the new restaurant with the correct BusinessTypeID
filter_query = {"BusinessName": "Penang Flavours"}

update_query = {"$set": {"BusinessTypeID": result["BusinessTypeID"]}}

update_result = collection.update_one(filter_query, update_query)

if update_result.modified_count > 0:
    print("New restaurant updated successfully!")
else:
    print("No update made. The restaurant might not exist or the update query did not match any documents.")

New restaurant updated successfully!


In [19]:
# Confirm that the new restaurant was updated
query = {"BusinessName": "Penang Flavours"}

updated_document = collection.find_one(query)

if updated_document:
    if updated_document.get("BusinessTypeID"):
        print("New restaurant was updated successfully!")
        print("Updated document:")
        pprint(updated_document)
    else:
        print("BusinessTypeID was not updated. Please verify.")
else:
    print("No document found for the given BusinessName.")

New restaurant was updated successfully!
Updated document:
{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('65f75f5cb92bfe5df041d440'),
 'geocode': {'latitude': '51.49014200', 'longitude': '0.08384000'},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [20]:
# Find how many documents have LocalAuthorityName as "Dover"
query = {"LocalAuthorityName": "Dover"}

count = collection.count_documents(query)

print("Number of documents with LocalAuthorityName as 'Dover':", count)

Number of documents with LocalAuthorityName as 'Dover': 994


In [21]:
# Delete all documents where LocalAuthorityName is "Dover"
query = {"LocalAuthorityName": "Dover"}

delete_result = collection.delete_many(query)

if delete_result.deleted_count > 0:
    print("Deleted", delete_result.deleted_count, "documents where LocalAuthorityName is 'Dover'.")
else:
    print("No documents found for deletion.")

Deleted 994 documents where LocalAuthorityName is 'Dover'.


In [22]:
# Check if any remaining documents include Dover
query = {"LocalAuthorityName": "Dover"}

count = collection.count_documents(query)

if count > 0:
    print("There are still", count, "documents remaining with LocalAuthorityName as 'Dover'.")
else:
    print("There are no remaining documents with LocalAuthorityName as 'Dover'.")

There are no remaining documents with LocalAuthorityName as 'Dover'.


In [23]:
# Check that other documents remain with 'find_one'
remaining_document = collection.find_one()

if remaining_document:
    print("Other documents remain in the collection:")
    pprint(remaining_document)
else:
    print("No other documents remain in the collection.")

Other documents remain in the collection:
{'AddressLine1': 'Wear Bay Road',
 'AddressLine2': 'Folkestone',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'Wear Bay Bowls Club',
 'BusinessType': 'Pub/bar/nightclub',
 'BusinessTypeID': 7843,
 'ChangesByServerID': 0,
 'Distance': 4591.821311183521,
 'FHRSID': 647177,
 'LocalAuthorityBusinessID': 'PI/000041489',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6PY',
 'RatingDate': '2014-03-31T00:00:00',
 'RatingKey': 'fhrs_4_en-gb',
 'RatingValue': '4',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('65f75b080d813317aabf3197'),
 'geocode': {'latitude': '51.086058', 'longitude': '1.196408'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/647177',
            'rel': 'self'}],

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [24]:
# Change the data type from String to Decimal for longitude and latitude
update_operation = [
    {"$set": {"geocode.longitude": {"$toDouble": "$geocode.longitude"}}},
    {"$set": {"geocode.latitude": {"$toDouble": "$geocode.latitude"}}}
]

update_result = collection.update_many({}, update_operation)

if update_result.modified_count > 0:
    print("Updated", update_result.modified_count, "documents.")
else:
    print("No documents were updated.")

Updated 38786 documents.


Use `update_many` to convert `RatingValue` to integer numbers.

In [25]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

UpdateResult({'n': 4091, 'nModified': 4091, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [26]:
# Change the data type from String to Integer for RatingValue
update_operation = [
    {"$set": {"RatingValue": {"$toInt": "$RatingValue"}}}
]

update_result = collection.update_many({}, update_operation)

if update_result.modified_count > 0:
    print("Updated", update_result.modified_count, "documents.")
else:
    print("No documents were updated.")

Updated 34695 documents.


In [27]:
# Check that the coordinates and rating value are now numbers
document = collection.find_one()

if document:
    # Check data types of coordinates
    latitude = document.get('geocode', {}).get('latitude')
    longitude = document.get('geocode', {}).get('longitude')
    rating_value = document.get('RatingValue')

    latitude_is_number = isinstance(latitude, (int, float))
    longitude_is_number = isinstance(longitude, (int, float))
    rating_value_is_number = isinstance(rating_value, int)

    print("Latitude is a number:", latitude_is_number)
    print("Longitude is a number:", longitude_is_number)
    print("RatingValue is a number:", rating_value_is_number)
else:
    print("No document found in the collection.")

Latitude is a number: True
Longitude is a number: True
RatingValue is a number: True
